In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import sklearn
import sklearn.datasets

### Prepare Data

In [ ]:
# Read in the data and set up the input and output
diagnosis_30_df = pd.read_csv('data/diagnosis.csv')

X = diagnosis_df
y = diagnosis_df['Diagnosis'].map({'M': 1, 'B': 0})

In [ ]:
# Split test, train, and demo
from sklearn.model_selection import train_test_split

X_use, demo_input, y_use, y_demo = train_test_split(X, y, random_state=2, test_size=0.02)
X_train, X_test, y_train, y_test = train_test_split(X_use, y_use, random_state=3)

In [ ]:
# Save a copy of the demo input data as csv and as X_demo
demo_input.drop('Diagnosis', axis=1).to_csv('data/demo_input.csv', index=False)
X_demo = demo_input

In [ ]:
# Drop irrelevant columns from X sets
for dataframe in [X_train, X_test, X_demo]:
    dataframe.drop(['ID number', 'Diagnosis'], axis=1, inplace=True)

In [ ]:
# Save X_train to csv in order to scale demo data in app
X_train.to_csv('data/X_train.csv', index=False)

In [ ]:
# Scale X
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_demo_scaled = X_scaler.transform(X_demo)

In [ ]:
# Convert y to categorical
from keras.utils import to_categorical

y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_demo_categorical = to_categorical(y_demo)

### Train Model

In [ ]:
# Create deep neural network model
# Use 3 layers, 6 nodes each
from keras.models import Sequential
from keras.layers import Dense

deep_model = Sequential()
deep_model.add(Dense(units=6, activation='relu', input_dim=20))
deep_model.add(Dense(units=6, activation='relu'))
deep_model.add(Dense(units=2, activation='softmax'))
deep_model.summary()

In [ ]:
# Fit model to training data
deep_model.compile(optimizer='adam',
                   loss='categorical_crossentropy',
                   metrics=['accuracy'])

deep_model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    shuffle=True,
    verbose=2
)

In [ ]:
# Test test data with deep model
model_loss, model_accuracy = deep_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Test demo data with deep model
model_loss, model_accuracy = deep_model.evaluate(
    X_demo_scaled, y_demo_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Save model to use in demo
deep_model.save('models/')

In [ ]:
# Load model, test to make sure it works
from keras.models import load_model
loaded_model = load_model('models/diagnosis_model_9856.h5')

model_loss, model_accuracy = loaded_model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(f"Deep Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

In [ ]:
# Trimmed data
# Read in the data and set up the input and output
diagnosis_df = pd.read_csv('feature selection/selected_diagnosis_Correlation.csv')

X = diagnosis_df
y = diagnosis_df['Diagnosis'].map({'M': 1, 'B': 0})

In [ ]:
# Split test, train, and demo
from sklearn.model_selection import train_test_split

X_use, demo_input, y_use, y_demo = train_test_split(X, y, random_state=2, test_size=0.02)
X_train, X_test, y_train, y_test = train_test_split(X_use, y_use, random_state=3)

# Save a copy of the demo input data as csv and as X_demo
demo_input.drop('Diagnosis', axis=1).to_csv('data/demo_input_20.csv', index=False)
X_demo = demo_input

# Drop irrelevant columns from X sets
for dataframe in [X_train, X_test, X_demo]:
    dataframe.drop('Diagnosis', axis=1, inplace=True)
    
# Save X_train to csv in order to scale demo data in app
X_train.to_csv('data/X_train.csv_20', index=False)

# Scale X
from sklearn.preprocessing import StandardScaler
X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_demo_scaled = X_scaler.transform(X_demo)

# Convert y to categorical
from keras.utils import to_categorical

y_train_categorical = to_categorical(y_train)
y_test_categorical = to_categorical(y_test)
y_demo_categorical = to_categorical(y_demo)